In [37]:
%reload_ext autoreload
%autoreload 2

import sys
import os

path_to_project = os.path.abspath(os.path.join(os.getcwd(), '../'))    
sys.path.insert(1, os.path.join(path_to_project))

In [38]:
import itertools
import pandas as pd
from sklearn.preprocessing import StandardScaler
from src.utils import one_hot
from src.directory import data_dir, NHANES_preprocessed_filename
from src.estimators import aipw_estimator, unadjusted_DM_estimator, ipw_estimator, t_learner, s_learner, x_learner

In [111]:
# load df
NHANES_preprocessed_filepath = os.path.join(data_dir, NHANES_preprocessed_filename)
df = pd.read_csv(NHANES_preprocessed_filepath, index_col='SEQN')

In [112]:
df.dropna(how='any', inplace=True)
df = df[df['age']>=35]
df.rename(columns={'martial_status': 'marital_status'}, inplace=True)
print('# patients with complete data: ', len(df))

# patients with complete data:  2548


In [113]:
# define relevant features
# features
z_col = 'ambient_light'
t_col = 'sleep_deprivation'
y_cols = ['HTN', 'DBP', 'SBP']

# column types
categorical_cols = ['physical_activity', 'depression', 'ANTIDEPRESSANTS_ANXIOLYTICS', 'GLUCOCORTICOIDS', 'sleep_troubles',
       'sleep_deprivation', 'diabetes', 'smoker', 'race_ethnicity', 'gender', 'health_insurance', 'marital_status','CVD', 
       'smoker_hx', 'HTN']
numerical_cols = ['daily_sedentary', 'accelerometer', 'BMI', 'age', 'poverty_ratio', 'depression', 'yearly_alcohol', 'DBP', 'SBP']

# columns not to transform
all_cols = df.columns
untransformed_cols = [x for x in all_cols if x in [*y_cols, t_col, z_col]]

In [114]:
light_cutoff = 1 # nightly minute-mean summed light exposure
df[z_col] = df[z_col].apply(lambda x: 1 if x <= light_cutoff else x)
df[z_col] = df[z_col].apply(lambda x: 0 if x > light_cutoff else x)

In [115]:
'''# prune df
df.dropna(how='any', inplace=True)

## transform df
# apply scalers
scaler = StandardScaler()
numerical_transformation_cols = list(set(numerical_cols) - set(untransformed_cols))
df[numerical_transformation_cols] = scaler.fit_transform(df[numerical_transformation_cols])
'''
# make z col binary (indicator of recommended max lux value during sleep)
light_cutoff = 1 # nightly minute-mean summed light exposure
df[z_col] = df[z_col].apply(lambda x: 1 if x <= light_cutoff else x)
df[z_col] = df[z_col].apply(lambda x: 0 if x > light_cutoff else x)

# one-hot encode multiclass categoricals
multiclass_cols = df[categorical_cols].columns[df[categorical_cols].nunique() > 2].tolist() 
categorical_transformation_cols = list(set(multiclass_cols) - set(untransformed_cols))
df = one_hot(df, categorical_transformation_cols)
df.columns = df.columns.str.replace('.0', '')

# get df as float
df = df.astype(float)

# get covariates
x_cols = list(set(df.columns) - set([t_col, *y_cols, z_col]))

# update lists of variable type
all_cols = df.columns
categorical_cols = [x for x in df.columns if any([x.startswith(y) for y in categorical_cols])]

In [126]:
estimators = [unadjusted_DM_estimator, ipw_estimator, t_learner, s_learner, x_learner, aipw_estimator]
estimator_names = estimator_names = [x.__name__ for x in estimators]
pairs = [' vs '.join(x[::-1]) for x in itertools.combinations(df[t_col].unique().astype(int).astype(str), 2)]
index = pd.MultiIndex.from_product([estimator_names, pairs, y_cols], names=['estimator', 'pair','outcome'])
tau_results = pd.DataFrame(index=index, columns=['tau'])

In [127]:
for tau_estimator, outcome in itertools.product(estimators, y_cols):
    estimator_name = tau_estimator.__name__
    print(f'Estimating effect of {t_col} on {outcome} using {estimator_name}...')
    continuous_outcome = False if outcome == 'HTN' else True
    results = tau_estimator(df, 
                            treatment_var=t_col, 
                            outcome_var=outcome, 
                            covariates=x_cols,
                            continuous_outcome=continuous_outcome)
    
    for pair in results.keys():
        tau = results[pair]['tau']
        tau_results.loc[(estimator_name, pair, outcome)] = tau

Estimating effect of sleep_deprivation on HTN using unadjusted_DM_estimator...
Estimating effect of sleep_deprivation on DBP using unadjusted_DM_estimator...
Estimating effect of sleep_deprivation on SBP using unadjusted_DM_estimator...
Estimating effect of sleep_deprivation on HTN using ipw_estimator...
Estimating effect of sleep_deprivation on DBP using ipw_estimator...
Estimating effect of sleep_deprivation on SBP using ipw_estimator...
Estimating effect of sleep_deprivation on HTN using t_learner...
Estimating effect of sleep_deprivation on DBP using t_learner...
Estimating effect of sleep_deprivation on SBP using t_learner...
Estimating effect of sleep_deprivation on HTN using s_learner...
Estimating effect of sleep_deprivation on DBP using s_learner...
Estimating effect of sleep_deprivation on SBP using s_learner...
Estimating effect of sleep_deprivation on HTN using x_learner...
Estimating effect of sleep_deprivation on DBP using x_learner...
Estimating effect of sleep_deprivati

In [129]:
tau_results.reset_index(inplace=True)

In [131]:
tau_results[tau_results['outcome']=='HTN']

,estimator,pair,outcome,tau
0,unadjusted_DM_estimator,1 vs 0,HTN,-0.078861
3,unadjusted_DM_estimator,2 vs 0,HTN,-0.015747
6,unadjusted_DM_estimator,2 vs 1,HTN,0.063114
9,ipw_estimator,1 vs 0,HTN,0.005025
12,ipw_estimator,2 vs 0,HTN,-0.139962
15,ipw_estimator,2 vs 1,HTN,-0.144987
18,t_learner,1 vs 0,HTN,0.0
21,t_learner,2 vs 0,HTN,0.0
24,t_learner,2 vs 1,HTN,0.0
27,s_learner,1 vs 0,HTN,-0.010694
